<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/reports/collection_analysis_title_info_by_item_type_and_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import pandas as pd

base_url = "https://collection-analysis.cincy.pl/current_collection.json"

session = requests.session()

In [2]:
# find the title info for given titles that are in a specific language

sql = """\
select
  bib.bib_record_num,
  bib_record.language_code,
  bib_record.country_code,
  bib.item_types,
  bib.best_author,
  bib.best_title,
  bib.publish_year,
  bib.bib_level_callnumber,
  bib.indexed_subjects,
  bib.genres,
  count(item.item_record_id)
from
  bib_record
  join bib on bib.bib_record_id = bib_record.record_id
  join item on item.bib_record_num = bib.bib_record_num
where
  language_code = :language_code
  and bib.bib_record_id in (
    select
      bib.bib_record_id
    from
      bib,
      json_each(bib.item_types)
    where
      json_each.value = :item_types
  )
group by
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10
order by
  bib.bib_record_num
limit
  :limit offset :offset
"""

limit = 1000
offset = 0
result_rows = list()

while True:
  response = session.get(
      base_url,
      params={
          'sql': sql,
          'item_types': 'Juvenile Book',
          # 'language_code': 'spa'
          'language_code': 'spa',
          'limit': limit,
          'offset': offset
      }
  )

  data = json.loads(response.text)
  result_rows.extend(data['rows'])
  print('.', end='')
  offset+=limit

  if (len(data['rows']) < limit):
    result_columns = data['columns']
    print('done.')
    break

..done.


In [3]:
from google.colab.data_table import DataTable

DataTable(
  pd.DataFrame(data=result_rows, columns=result_columns)
)

,bib_record_num,language_code,country_code,item_types,best_author,best_title,publish_year,bib_level_callnumber,indexed_subjects,genres,count(item.item_record_id)
0,1012489,spa,sp,"[""Juvenile Book""]",,Cuentos rusos,1981.0,j398.4,"[""fairy tales""]",None,1
1,1012535,spa,sp,"[""Juvenile Book""]","Grimm, Jacob, 1785-1863.",Rapunzel,1981.0,j398.4,None,None,1
2,1013103,spa,sp,"[""Juvenile Book""]","Lionni, Leo, 1910-1999",Nadarín.,1963.0,s l easy,None,None,4
3,1013630,spa,sp,"[""Juvenile Book""]","Grimm, Jacob, 1785-1863.",La bella durmiente,1985.0,"398.4 g864 ths, 1985","[""fairy tales""]",None,1
4,1019037,spa,ilu,"[""Juvenile Book""]","Wylie, Joanne.",Has abrazado hoy a tu monstruo? : dale un beso...,1986.0,s j398.2,"[""juvenile literature monsters fiction"", ""mons...",None,1
...,...,...,...,...,...,...,...,...,...,...,...
1544,3815434,spa,nyu,"[""Juvenile Book""]","Otheguy, Emma, author.",Martina tiene muchas tías,2023.0,s o easy,"[""imagination juvenile fiction"", ""juvenile fic...","[""Picture books.""]",12
1545,3815454,spa,nyu,"[""Juvenile Book""]","Leisink, Carolijn, author.",Una fiesta para abuela / escrito por Carolijn ...,2023.0,s l easy,"[""cooking surinamese juvenile fiction"", ""juven...",None,12
1546,3818288,spa,mau,"[""Juvenile Book""]","Sánchez Vegara, Ma Isabel (María Isabel), author.",Martin Luther King Jr. : en español,2023.0,s 323.092 k53 zs 2023,"[""king martin luther jr 1929 1968 juve...","[""Picture books."", ""Biographies.""]",7
1547,3820973,spa,nyu,"[""Juvenile Book""]","Carpenter, Mikala, author.",Places = Lugares,2023.0,s c easy,"[""city and town life juvenile fiction"", ""juven...","[""Board books.""]",12
